In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer


In [ ]:
# Load the news dataset
news_data = pd.read_csv("../data/raw_analyst_ratings.csv")
print(news_data.head())
print(news_data.info())




In [ ]:
# Example: Analyze headline length
news_data['headline_length'] = news_data['headline'].str.len()
sns.histplot(news_data['headline_length'], bins=30, kde=True)
plt.show()


In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
import pandas as pd

# Initialize the sentiment analyzer
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

# Calculate sentiment scores
news_data['sentiment'] = news_data['headline'].apply(lambda x: sia.polarity_scores(x)['compound'])

# Categorize sentiment as positive, neutral, or negative
news_data['sentiment_category'] = news_data['sentiment'].apply(
    lambda score: 'positive' if score > 0.05 else ('negative' if score < -0.05 else 'neutral')
)

# Display sentiment distribution
print(news_data['sentiment_category'].value_counts())


In [ ]:

sns.countplot(data=news_data, x='sentiment_category', palette='viridis')
plt.title('Sentiment Distribution')
plt.xlabel('Sentiment')
plt.ylabel('Number of Headlines')
plt.show()


In [ ]:
#  Extract Common Keywords or Phrases

# Initialize TF-IDF vectorizer
tfidf = TfidfVectorizer(stop_words='english', max_features=20)

# Fit and transform headlines
tfidf_matrix = tfidf.fit_transform(news_data['headline'])
top_keywords = tfidf.get_feature_names_out()

print("Top Keywords:", top_keywords)


In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

# Initialize LDA model
lda = LatentDirichletAllocation(n_components=5, random_state=42)  # 5 topics
lda.fit(tfidf_matrix)

# Display top words for each topic
for idx, topic in enumerate(lda.components_):
    print(f"Topic {idx + 1}:")
    print([tfidf.get_feature_names_out()[i] for i in topic.argsort()[-10:]])


In [ ]:
# save the result 
news_data.to_csv("../data/processed_sentiment_topic_modeling.csv", index=False)


In [ ]:
# topic modeling

import os


yfinance_data_dir = '../data/yfinance_data/'
csv_files = [f for f in os.listdir(yfinance_data_dir) if f.endswith('.csv')]
stock_data = {file: pd.read_csv(os.path.join(yfinance_data_dir, file)) for file in csv_files}
print(stock_data['AAPL_historical_data.csv'].head())  # Checking the data for AAPL


In [ ]:
news_data['date'] = pd.to_datetime(news_data['date'], errors='coerce')
print(news_data['date'].isnull().sum())  # Check for invalid entries


In [ ]:
print(news_data['date'].dtypes)
print(news_data['date'].head())


In [ ]:
news_data['publication_date'] = news_data['date'].dt.date
daily_publication_counts = news_data['publication_date'].value_counts().sort_index()

import matplotlib.pyplot as plt

plt.figure(dpi= 200, figsize=(12, 6))
daily_publication_counts.plot(kind='line', title="Daily Publication Trends")
plt.xlabel("Date")
plt.ylabel("Number of Articles")
plt.grid()
plt.show()


In [ ]:
news_data.head()

In [ ]:
publisher_counts = news_data['publisher'].value_counts()
print(publisher_counts.head(10))  # Display the top 10 publishers


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

top_publishers = publisher_counts.head(10)
plt.figure(figsize=(12, 6))
sns.barplot(x=top_publishers.index, y=top_publishers.values, palette="viridis")
plt.xticks(rotation=45)
plt.title("Top 10 Publishers by Article Count")
plt.xlabel("Publisher")
plt.ylabel("Number of Articles")
plt.show()


In [ ]:
for publisher in top_publishers.index:
    print(f"\nHeadlines for {publisher}:")
    print(news_data[news_data['publisher'] == publisher]['headline'].head(5))  # Show 5 sample headlines


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Analyze content for a specific publisher
selected_publisher = top_publishers.index[0]  # Example: the most frequent publisher
publisher_headlines = news_data[news_data['publisher'] == selected_publisher]['headline']

tfidf = TfidfVectorizer(stop_words='english', max_features=20)
tfidf_matrix = tfidf.fit_transform(publisher_headlines)
print(f"Top Keywords for {selected_publisher}: {tfidf.get_feature_names_out()}")


In [ ]:
news_data['publisher_domain'] = news_data['publisher'].apply(
    lambda x: x.split('@')[-1] if '@' in x else None
)
domain_counts = news_data['publisher_domain'].value_counts()
print(domain_counts.head(10))  # Display the top 10 domains



In [ ]:
top_domains = domain_counts.head(10)
plt.figure(figsize=(12, 6))
sns.barplot(x=top_domains.index, y=top_domains.values, palette="coolwarm")
plt.xticks(rotation=45)
plt.title("Top 10 Email Domains by Article Count")
plt.xlabel("Domain")
plt.ylabel("Number of Articles")
plt.show()
